In [1]:
import time
import numpy as np
import torch
import torchvision
from torch.autograd import Variable
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# Constants
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
COLOR_CHANNELS = 3
EPOCHS = 300
LEARNING_RATES = [.00001, 0.0001, 0.001, 0.01, 0.1]
KEEP_RATES = [.5, .65, .8]
MOMENTUM_RATES = [.25, .5, .75]
WEIGHT_DECAY_RATES = [.0005, .005, .05]
BATCH_SIZE = 32
BATCH_IMAGE_COUNT = 10000
TRAIN_BATCHS = ["data_batch_1", "data_batch_2", "data_batch_3", "data_batch_4"]
TEST_BATCHES = ["data_batch_5"]
CLASSES = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
N_CLASSES = len(CLASSES)
PLOT = False

In [3]:
class Net(torch.nn.Module):
    def __init__(self, n_hidden_nodes, n_hidden_layers, activation, keep_rate=0):
        super(Net, self).__init__()
        self.n_hidden_nodes = n_hidden_nodes
        self.n_hidden_layers = n_hidden_layers
        self.activation = activation
        if not keep_rate:
            keep_rate = 0.5
        self.keep_rate = keep_rate
        # Set up perceptron layers and add dropout
        self.fc1 = torch.nn.Linear(IMAGE_WIDTH * IMAGE_WIDTH * COLOR_CHANNELS,
                                   n_hidden_nodes)
        self.fc1_drop = torch.nn.Dropout(1 - keep_rate)
        if n_hidden_layers == 2:
            self.fc2 = torch.nn.Linear(n_hidden_nodes,
                                       n_hidden_nodes)
            self.fc2_drop = torch.nn.Dropout(1 - keep_rate)

        self.out = torch.nn.Linear(n_hidden_nodes, N_CLASSES)

    def forward(self, x):
        x = x.view(-1, IMAGE_WIDTH * IMAGE_WIDTH * COLOR_CHANNELS)
        if self.activation == "sigmoid":
            sigmoid = torch.nn.Sigmoid()
            x = sigmoid(self.fc1(x))
        elif self.activation == "relu":
            x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc1_drop(x)
        if self.n_hidden_layers == 2:
            if self.activation == "sigmoid":
                x = sigmoid(self.fc2(x))
            elif self.activation == "relu":
                x = torch.nn.functional.relu(self.fc2(x))
            x = self.fc2_drop(x)
        return torch.nn.functional.log_softmax(self.out(x))

In [4]:
def train(epoch, model, train_loader, optimizer, log_interval=100, cuda=None):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)

        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()
        accuracy = 100. * correct / len(train_loader.dataset)
        loss = torch.nn.functional.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        '''
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} Accuracy: {}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0], accuracy))
        '''

In [5]:
def validate(loss_vector, accuracy_vector, model, validation_loader, cuda=None):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        val_loss += torch.nn.functional.nll_loss(output, target).data
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)

    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))

In [8]:
def main():
    cuda = torch.cuda.is_available()
    print('Using PyTorch version:', torch.__version__, 'CUDA:', cuda)

    transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                              shuffle=True, num_workers=0, pin_memory=False)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    validation_loader = torch.utils.data.DataLoader(testset, batch_size=4,
                                             shuffle=False, num_workers=0, pin_memory=False)

    hidden_nodes = 100
    layers = 1
    model = Net(hidden_nodes, layers, "sigmoid")
    if cuda:
        model.cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATES[1])

    loss_vector = []
    acc_vector = []
    for epoch in range(1, EPOCHS + 1):
        print('Epoch {}'.format(epoch))
        train(epoch, model, train_loader, optimizer, cuda=cuda)
        validate(loss_vector, acc_vector, model, validation_loader, cuda=cuda)
        if epoch == 25:
            break

In [9]:
main()

Using PyTorch version: 1.12.1 CUDA: False
Files already downloaded and verified
Files already downloaded and verified
Epoch 1


/var/folders/lx/cv57zfl536v865_7hwvz7vsh0000gn/T/ipykernel_8585/2398099917.py:35: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return torch.nn.functional.log_softmax(self.out(x))
/var/folders/lx/cv57zfl536v865_7hwvz7vsh0000gn/T/ipykernel_8585/4101122291.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Validation set: Average loss: 2.2055, Accuracy: 2685/10000 (27%)

Epoch 2

Validation set: Average loss: 2.1463, Accuracy: 2856/10000 (29%)

Epoch 3

Validation set: Average loss: 2.1028, Accuracy: 2976/10000 (30%)

Epoch 4

Validation set: Average loss: 2.0683, Accuracy: 3026/10000 (30%)

Epoch 5

Validation set: Average loss: 2.0404, Accuracy: 3043/10000 (30%)

Epoch 6

Validation set: Average loss: 2.0174, Accuracy: 3126/10000 (31%)

Epoch 7

Validation set: Average loss: 1.9980, Accuracy: 3120/10000 (31%)

Epoch 8

Validation set: Average loss: 1.9816, Accuracy: 3170/10000 (32%)

Epoch 9

Validation set: Average loss: 1.9675, Accuracy: 3201/10000 (32%)

Epoch 10

Validation set: Average loss: 1.9554, Accuracy: 3226/10000 (32%)

Epoch 11

Validation set: Average loss: 1.9446, Accuracy: 3248/10000 (32%)

Epoch 12

Validation set: Average loss: 1.9350, Accuracy: 3317/10000 (33%)

Epoch 13

Validation set: Average loss: 1.9265, Accuracy: 3318/10000 (33%)

Epoch 14

Validation set: Ave